In [34]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import warnings
from sklearn import preprocessing,model_selection,metrics
from lightgbm import LGBMRanker
from tabulate import tabulate
from functools import wraps

data = pd.read_csv('/home/jovyan/shares/SR004.nfs2/amaksimova/exp/ranker/data/train_data.csv')
data

,Unnamed: 0.2,Unnamed: 0.1,index,Unnamed: 0,vacancy_id,user_id,relavence_score,mean_names,mean_desc,user_salary_from,user_salary_to,vacancy_salary_from,vacancy_salary_to,intersection_keyskill
0,23,23,23,23,v_968261,u_513709,0.33,1.000000,1.000000,70000.000000,0.0,70000.0,NaN,1.000000
1,24,24,24,24,v_968261,u_513709,0.33,1.000000,1.000000,70000.000000,0.0,70000.0,NaN,1.000000
2,25,25,25,25,v_968261,u_513709,1.00,1.000000,1.000000,70000.000000,0.0,70000.0,NaN,1.000000
3,42,42,42,42,v_128856,u_1070284,0.33,1.000000,1.000000,0.000000,0.0,NaN,NaN,1.000000
4,43,43,43,43,v_128856,u_1070284,1.00,1.000000,1.000000,0.000000,0.0,NaN,NaN,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30580,48771,48771,48771,48771,v_915763,u_302844,0.66,0.519392,0.691904,39722.222222,45875.0,37500.0,46000.0,1.000000
30581,48772,48772,48772,48772,v_1726400,u_302844,0.33,0.369177,0.653639,39722.222222,45875.0,46000.0,69000.0,1.666667
30582,48773,48773,48773,48773,v_1902404,u_302844,0.33,0.581152,0.666719,39722.222222,45875.0,25000.0,NaN,1.166667
30583,48774,48774,48774,48774,v_56015,u_302844,0.33,0.404525,0.631718,39722.222222,45875.0,NaN,25000.0,0.000000


In [35]:
data.isna().sum()

Unnamed: 0.2                 0
Unnamed: 0.1                 0
index                        0
Unnamed: 0                   0
vacancy_id                   0
user_id                      0
relavence_score              0
mean_names                   0
mean_desc                    0
user_salary_from             0
user_salary_to               0
vacancy_salary_from      10063
vacancy_salary_to        17510
intersection_keyskill        0
dtype: int64

In [36]:
data['vacancy_salary_from'] = data['vacancy_salary_from'].fillna((data['vacancy_salary_from'].mean()))
data['vacancy_salary_to'] = data['vacancy_salary_to'].fillna((data['vacancy_salary_to'].mean()))
data

,Unnamed: 0.2,Unnamed: 0.1,index,Unnamed: 0,vacancy_id,user_id,relavence_score,mean_names,mean_desc,user_salary_from,user_salary_to,vacancy_salary_from,vacancy_salary_to,intersection_keyskill
0,23,23,23,23,v_968261,u_513709,0.33,1.000000,1.000000,70000.000000,0.0,70000.000000,122686.091013,1.000000
1,24,24,24,24,v_968261,u_513709,0.33,1.000000,1.000000,70000.000000,0.0,70000.000000,122686.091013,1.000000
2,25,25,25,25,v_968261,u_513709,1.00,1.000000,1.000000,70000.000000,0.0,70000.000000,122686.091013,1.000000
3,42,42,42,42,v_128856,u_1070284,0.33,1.000000,1.000000,0.000000,0.0,87338.856544,122686.091013,1.000000
4,43,43,43,43,v_128856,u_1070284,1.00,1.000000,1.000000,0.000000,0.0,87338.856544,122686.091013,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30580,48771,48771,48771,48771,v_915763,u_302844,0.66,0.519392,0.691904,39722.222222,45875.0,37500.000000,46000.000000,1.000000
30581,48772,48772,48772,48772,v_1726400,u_302844,0.33,0.369177,0.653639,39722.222222,45875.0,46000.000000,69000.000000,1.666667
30582,48773,48773,48773,48773,v_1902404,u_302844,0.33,0.581152,0.666719,39722.222222,45875.0,25000.000000,122686.091013,1.166667
30583,48774,48774,48774,48774,v_56015,u_302844,0.33,0.404525,0.631718,39722.222222,45875.0,87338.856544,25000.000000,0.000000


In [37]:
data.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'index', 'Unnamed: 0', 'vacancy_id',
       'user_id', 'relavence_score', 'mean_names', 'mean_desc',
       'user_salary_from', 'user_salary_to', 'vacancy_salary_from',
       'vacancy_salary_to', 'intersection_keyskill'],
      dtype='object')

In [38]:
features = [ 'mean_names', 'mean_desc',
       'user_salary_from', 'user_salary_to', 'vacancy_salary_from',
       'vacancy_salary_to', 'intersection_keyskill']
target = 'relavence_score'

In [39]:
data = data.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1', 'index', 'Unnamed: 0', 'vacancy_id'])
data

,user_id,relavence_score,mean_names,mean_desc,user_salary_from,user_salary_to,vacancy_salary_from,vacancy_salary_to,intersection_keyskill
0,u_513709,0.33,1.000000,1.000000,70000.000000,0.0,70000.000000,122686.091013,1.000000
1,u_513709,0.33,1.000000,1.000000,70000.000000,0.0,70000.000000,122686.091013,1.000000
2,u_513709,1.00,1.000000,1.000000,70000.000000,0.0,70000.000000,122686.091013,1.000000
3,u_1070284,0.33,1.000000,1.000000,0.000000,0.0,87338.856544,122686.091013,1.000000
4,u_1070284,1.00,1.000000,1.000000,0.000000,0.0,87338.856544,122686.091013,1.000000
...,...,...,...,...,...,...,...,...,...
30580,u_302844,0.66,0.519392,0.691904,39722.222222,45875.0,37500.000000,46000.000000,1.000000
30581,u_302844,0.33,0.369177,0.653639,39722.222222,45875.0,46000.000000,69000.000000,1.666667
30582,u_302844,0.33,0.581152,0.666719,39722.222222,45875.0,25000.000000,122686.091013,1.166667
30583,u_302844,0.33,0.404525,0.631718,39722.222222,45875.0,87338.856544,25000.000000,0.000000


In [40]:
data[target] = [int(i*10) for i in list(data[target])]

In [41]:
from sklearn.model_selection import train_test_split
data.sample(frac=1)
train, test = train_test_split(data, test_size=0.1)

In [42]:


xtrain,xtest,ytrain,ytest = train.drop(columns=target), test.drop(columns=target), train[target], test[target]

In [43]:
get_group_size = lambda df: df.reset_index().groupby("user_id")['user_id'].count()
train_groups = get_group_size(xtrain)
test_groups = get_group_size(xtest)

In [47]:
model = LGBMRanker(objective="lambdarank",n_estimators=100)
model.fit(xtrain.drop(columns='user_id'),ytrain,group=train_groups,eval_set=[(xtest.drop(columns='user_id'),ytest)],eval_group=[test_groups],eval_metric=['map'])

[LightGBM] [Info] Total groups: 2078, total data: 27526
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000540 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1546
[LightGBM] [Info] Number of data points in the train set: 27526, number of used features: 7
[LightGBM] [Info] Total groups: 1224, total data: 3059


LGBMRanker(objective='lambdarank')

In [56]:
import joblib
# save model
joblib.dump(model, 'LGBMRanker.pkl')
# load model
gbm_pickle = joblib.load('LGBMRanker.pkl')

In [58]:
tmp = pd.DataFrame()
tmp['mean_names'] = [0.2,0.2]
tmp['mean_desc'] = [0.2,0.2]
tmp['user_salary_from'] = [15000,2222]
tmp['user_salary_to'] = [80000,222222]
tmp['vacancy_salary_from'] = [150000,22222]
tmp['vacancy_salary_to'] = [500000,22222]
tmp['intersection_keyskill'] = [0,2]
tmp

,mean_names,mean_desc,user_salary_from,user_salary_to,vacancy_salary_from,vacancy_salary_to,intersection_keyskill
0,0.2,0.2,15000,80000,150000,500000,0
1,0.2,0.2,2222,222222,22222,22222,2


In [59]:
gbm_pickle.predict(tmp)

array([-2.81044555, -1.82028307])